In [1]:
import pandas as pd
import folium


In [2]:
df = pd.read_csv("./buildings_annarbor_coordinates.csv") #reading the csv form the second notebook, with coordinates.

In [3]:
df.head(5)

,index,Index,Number,Street Name,Details,Region,city,state,Address,Latitude,Longitude,Zipcode,Country,Link_Address,Link_Coordinates
0,0,1,109,East Ann Street,Hoban Block,D,Ann Arbor,Michigan,"109 East Ann Street, Ann Arbor, Michigan",42.282549,-83.747928,48104,United States,https://www.google.com/maps?q=109%20East%20Ann...,"https://maps.google.com/?q=42.282549,-83.747928"
1,1,1,110,East Ann Street,Hoban Block,D,Ann Arbor,Michigan,"110 East Ann Street, Ann Arbor, Michigan",42.282346,-83.748343,48104,United States,https://www.google.com/maps?q=110%20East%20Ann...,"https://maps.google.com/?q=42.2823463,-83.7483426"
2,2,1,111,East Ann Street,Hoban Block,D,Ann Arbor,Michigan,"111 East Ann Street, Ann Arbor, Michigan",42.282541,-83.747850,48104,United States,https://www.google.com/maps?q=111%20East%20Ann...,"https://maps.google.com/?q=42.28254099999999,-..."
3,3,1,112,East Ann Street,Hoban Block,D,Ann Arbor,Michigan,"112 East Ann Street, Ann Arbor, Michigan",42.282345,-83.748316,48104,United States,https://www.google.com/maps?q=112%20East%20Ann...,"https://maps.google.com/?q=42.2823455,-83.7483163"
4,4,1,113,East Ann Street,Hoban Block,D,Ann Arbor,Michigan,"113 East Ann Street, Ann Arbor, Michigan",42.282516,-83.747761,48104,United States,https://www.google.com/maps?q=113%20East%20Ann...,"https://maps.google.com/?q=42.2825159,-83.7477..."


In [6]:

# Initialize map, from folium.
m = folium.Map(location=[df['Latitude'][0], df['Longitude'][0]], zoom_start=12)


# Add ESRI Satellite basemap
esri_satellite = folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='Esri Satellite',
    overlay=False,
    control=True
).add_to(m)

# Add Google Hybrid basemap
google_hybrid = folium.TileLayer(
    tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
    attr='Google',
    name='Google Hybrid',
    overlay=False,
    control=True
).add_to(m)

#add Google maps
google_maps = folium.TileLayer(
    tiles='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
    attr='Google',
    name='Google Maps',
    overlay=False,
    control=True
).add_to(m)

# Define color mapping for regions
color_dict = {
    'D': 'red',
    'E': 'blue',
    'N': 'green',
    'W': 'orange'
}

# Create a FeatureGroup for each region
feature_groups = {region: folium.FeatureGroup(name=f'Region {region}') for region in color_dict.keys()}


# Add markers to the corresponding FeatureGroup
for i, row in df.iterrows():
    popup_text = f"""
    <div style="width: 200px; 
                height: auto; 
                background-color: white; 
                border: 2px solid grey; 
                border-radius: 5px; 
                padding: 10px;">
        <b>Address:</b> {row['Address']}<br>
        <b>Details:</b> {row['Details']}<br>
        <b>Direction:</b> <a href="{row['Link_Address']}" target="_blank">Open in Maps</a>
    </div>
    """
    marker_color = color_dict.get(row['Region'], 'black')  # Default to black if Region is not in color_dict
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=popup_text,
        icon=folium.Icon(color=marker_color,  icon="star", prefix='fa')#,  icon_size=(20, 15)
    ).add_to(feature_groups[row['Region']])
# Create a legend HTML
legend_html = '''
     <div style="position: fixed; 
     bottom: 50px; left: 50px; width: 200px; height: 150px; 
     background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
     ">&nbsp; <b>Legend</b> <br>
     &nbsp; <i class="fa fa-map-marker fa-2x" style="color:red"></i>&nbsp; Region D (Downtown) <br>
     &nbsp; <i class="fa fa-map-marker fa-2x" style="color:blue"></i>&nbsp; Region E (East) <br>
     &nbsp; <i class="fa fa-map-marker fa-2x" style="color:green"></i>&nbsp; Region N (North) <br>
     &nbsp; <i class="fa fa-map-marker fa-2x" style="color:orange"></i>&nbsp; Region W (West) <br>
      </div>
     '''

# Add legend to map
m.get_root().html.add_child(folium.Element(legend_html))



# Add all FeatureGroups to the map
for fg in feature_groups.values():
    fg.add_to(m)


# Add LayerControl to the map
folium.LayerControl().add_to(m)

m

In [7]:
m.save("buildings_ann_Arbor.html")